In [3]:
import schnetpack as spk
import torch
import schnetpack.nn 
import schnetpack.data
import pandas as pd
import scipy.linalg as la
from schnetpack.datasets import QM9
import copy





In [6]:
#Intermediare embedding output pipeline

#Load last checkpoint file from from trained model
checkpoint_path = './trained_models/qm9i5e100/checkpoints/checkpoint-10.pth.tar'
load_checkpoint = torch.load(checkpoint_path)

#Load the Best model where it is stored after training
best_model = torch.load('./trained_models/qm9i5e300/best_model')
model = best_model

                   

device='cpu'

# Download QM9 dataset to use in evaluating model
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

# Load split file for the train, validation and test 
train, val, test = spk.data.train_test_split(qm9data, split_file='./trained_models/qm9i5e300/split.npz')


#set up device and atoms converter for input

converter = spk.data.AtomsConverter(device=device)

test_loader = spk.AtomsLoader(test, batch_size=100)
converter = spk.data.AtomsConverter(device=device)

at, props = qm9data.get_properties(idx=0)

calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.U0)
at.set_calculator(calculator)


inputs = converter(at)

model(inputs)


from schnetpack.representation import SchNet

model = SchNet()
y = model.state_dict()

del y['embedding.weight']



del model.load_state_dict(load_checkpoint,strict=False)['embedding.weight']


print(x)

#print(x)




#print(model.state_dict()['output_modules.0.atomref.weight'])

print('Prediction:', at.get_total_energy())


TypeError: '_IncompatibleKeys' object does not support item deletion

In [ ]:
for param in model.parameters():
    print(param)

In [ ]:

#We choose None to instatiate the variable originally
embedding_output=None
#Define a hooking function that will fetch the out_tensor 
#from the layer of interest given an input tensor
def embedding_hook(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global embedding_output
    #Update the embedding_output variable to be equal to our output tensor
    embedding_output=out_tensor

#Hook our function to the embedding layer during the forward pass
model.representation.embedding.register_forward_hook(embedding_hook)

#Check if None
print(embedding_output)

#Forward pass of the tensor inputs
model(inputs)

#embedding_output SHOULD be changed
print(embedding_output)

In [ ]:


interactions_output=None

def interactions_hook(self, inp_tensor, out_tensor):
    global interactions_output
    interactions_output=out_tensor

model.representation.interactions\.1\.dense.biasregister_forward_hook(interactions_hook)

#Check if None
print(interactions_output)

#Forward pass of the tensor inputs
model(inputs)

#embedding_output SHOULD be changed
print(interactions_output)

In [ ]:
"""
1) Write hook functions for each layer in the model and compile a database of the intermediare outputs.


2) Store these outputs into a h5 file using h5py
    How does h5 work?
        - create a dictionary of dataset
            dataset = h5py.File("name.h5", "w")
            dataset.create_dataset("name_dataset", data=np.array)
            ...
            dataset.close()
            dataset["name_dataset"]

3) Small toy systems of interest where we can look at specific chemical groups
    3.1a) Carboxylic acids
    3.2a) Ketone
    3.3a) Aldehyde
    3.4a) Alcohols
    3.5a) Ethers
    
    3.1b) Amine
    3.2b) Amide
    3.3b) Azo
    3.4b) Nitro
    ...
    
    Contrast chemically similar groups that are distinguished by atom type
    
4) Construct descriptors to differentiate between different chemical functionalities
    4.1) PCA
    4.2) t-SNE
    4.3) Self-Organising maps
    4.4) Autoencoders
"""